In [1]:
import pandas as pd
import numpy as np
import h5py
import math

from keras.models import Sequential, model_from_json
from keras.layers import Dense, Dropout, Activation, regularizers, Flatten
from keras.callbacks import CSVLogger

from sklearn import ensemble, preprocessing, multiclass
from sklearn.preprocessing import Imputer
from sklearn.model_selection import cross_val_score, train_test_split

from collections import Counter

Using TensorFlow backend.


In [2]:
# Transformation

def sqrt(col):
    return list(map(np.sqrt, col));

def freq(col):
    col = np.floor(col)
    counter = Counter(col)
    return [counter.get(elem) for elem in col]

def tanh(col):
    return list(map(np.tanh, col));

def log(col):
    return list(map(np.log, col));

def my_sigmoid(x):
    return 1 / (1 + math.exp(-x))

def sigmoid(col):
    return list(map(my_sigmoid, col))

def square(col):
    return list(map(np.square, col))

def normalize_val(num, col_min, col_max, norm_range):
    width = col_max-col_min

    if(width == 0): width = 1

    return (num-col_min)/width * (norm_range[1]-norm_range[0]) + norm_range[0]

def normalize(col):
    norm_range = (-1,1)
    col_max = np.amax(col)
    col_min = np.amin(col)
    
    return list(map(lambda x : normalize_val(x, col_min, col_max, norm_range), col))
    


In [114]:
# Globals

# Datasets
#dids = np.load("datasets/indexes.npy")
dids = np.load("magicindexes.npy")

# RF model parameters
seed = 67
#transformations = [sqrt, freq]
#transformations_name = ["sqrt", "freq"]
transformations = [tanh, log, sigmoid]
transformations_name = ["tanh", "log", "sigmoid"]
trans2target1 = {}
trans2target2 = {}
trans2target3 = {}

# Comrpessed Dataset paramters
qsa_representation = []
num_bin = 10
too_big = 10000

# Neural Nets Parameters and Variables
MLP_LFE_Nets = {}
inp_shape = (2,num_bin)
dropout = 0.2
norm = (0, 10)
pred_threshold = 0.51
train_set_max = 80000
test_set_max = 80000


In [4]:
#def binarize_dataset():

def load_dataset(id):
    X = np.load("datasets/binary_numeric/" + str(id) + "-data.npy")
    y = np.load("datasets/binary_numeric/" + str(id) + "-target.npy")
    categorical = np.load("datasets/binary_numeric/" + str(id) + "-categorical.npy")
    return X,y,categorical

    
def evaluate_model(X, y, categorical):
    imp = Imputer(missing_values="NaN")
    X = imp.fit_transform(X)
    enc = preprocessing.OneHotEncoder(categorical_features=categorical)
    X = enc.fit_transform(X)
    clf = ensemble.RandomForestClassifier(random_state=seed)
    #clf_ovsr = multiclass.OneVsRestClassifier(clf, n_jobs=-1)
    
    return cross_val_score(clf, X, y,cv=10)
    
def is_positive(X,y,categorical,base_score,transformation,feature):
    transformed_feature = np.array(transformation(X[:,feature]))
    X = np.c_[X,transformed_feature]
    categorical = np.append(categorical,False)
    new_score = evaluate_model(X,y,categorical).mean()
    
    return 1 if(base_score <= (new_score - 0.01)) else 0

def is_positive_2(X, y, categorical, base_score, transformation, feature):
    transformed_feature = np.array(transformation(X[:,feature]))
    new_score = evaluate_model(transformed_feature.reshape(-1,1),y,[False]).mean()
    
    return 1 if(base_score <= (new_score - 0.005)) else 0

def is_positive_3(X, y, categorical, base_score, transformation, feature):
    transformed_feature = np.array(transformation(X[:,feature]))
    new_score = evaluate_model(transformed_feature.reshape(-1,1),y,[False]).mean()
    
    return 1 if(new_score > base_score*1.01) else 0
    
    

In [5]:
# Build the target for the compressed feature
bad_datasets = []

def build_target_for_compressed(dids):

    for transf in transformations:
        trans2target1[transf] = []
        trans2target2[transf] = []
        trans2target3[transf] = []

    
    for did in dids:
        print("Start dataset number", did)
        
        try:

            X, y, categorical = load_dataset(did)       

            new_indexes = []

            if(X.shape[0] > too_big):
                new_indexes = np.random.choice(X.shape[0], too_big, replace=False)
                X = X[new_indexes]
                y = y[new_indexes]

            base_score = evaluate_model(X, y, categorical).mean()

            # Find the indexes of numeric attributes
            numerical_indexes = np.where(np.invert(categorical))[0]
            sample_numerical_indexes = np.random.choice(numerical_indexes, min(numerical_indexes.shape[0],10), replace=False)

            for i,transf in enumerate(transformations):
                for feature in sample_numerical_indexes:

                    print("\tEvaluating feature " + str(feature))

                    mlp_target_1 = is_positive(X,y,categorical,base_score,transf, feature)
                    mlp_target_2 = is_positive_2(X,y,categorical,base_score,transf, feature)
                    mlp_target_3 = is_positive_3(X,y,categorical,base_score,transf, feature)

                    print("\t\t" + str(mlp_target_1), str(mlp_target_2), str(mlp_target_3))

                    trans2target1[transf].append((did,feature,mlp_target_1))
                    trans2target2[transf].append((did,feature,mlp_target_2))
                    trans2target3[transf].append((did,feature,mlp_target_3))

        except:
                print("The evaluation of dataset " + str(did) + " failed")
                bad_datasets.append(did)
                continue

In [6]:
# Save the result
def save_target_for_compressed(path):

    for transf, name in zip(transformations, transformations_name):
        np.save(path + name + "1", trans2target1[transf])
        np.save(path + name + "2", trans2target2[transf])
        np.save(path + name + "3", trans2target3[transf])


In [7]:
def normalize_Rx(matrix):
    
    Rxc = np.zeros(shape=matrix.shape)
    
    for i,row in enumerate(matrix):
        max_c = np.amax(row)
        min_c = np.amin(row)
        bin_width = (max_c-min_c)/(norm[1]-norm[0])
        Rxc[i] = np.apply_along_axis(lambda x : np.floor((x-min_c)/(bin_width)+norm[0]), 0, row)
    
    return Rxc

def to_quantile_sketch_array(did, col, targets, bins, t_class, index):
    max_c = np.nanmax(col)
    min_c = np.nanmin(col)
    bin_width = (max_c-min_c)/num_bin
    Rx = np.zeros(shape=(2,num_bin))
    
    if(bin_width == 0):
        return
    
    for val,y in zip(col,targets):
        if not np.isnan(val):
            bin_value = int(np.floor((val-min_c)/bin_width))
            bin_value = np.clip(bin_value, 0, num_bin-1)
            my_class = 0 if t_class == y else 1
            Rx[my_class][bin_value] = Rx[my_class][bin_value] + 1
            
    Rx = normalize_Rx(Rx)

    qsa_representation.append(np.insert(Rx.flatten(), 0, [did,index]))

In [8]:
# Build the compressed dataset
def build_compressed_dataset(dids):

    for did in dids:
        print("Start dataset number", did)

        try:
            X, y, categorical = load_dataset(did)
        
            if(X.shape[0] > too_big):
                new_indexes = np.random.choice(X.shape[0], too_big, replace=False)
                X = X[new_indexes]
                y = y[new_indexes]

            numerical_indexes = np.where(np.invert(categorical))[0]

            classes = set(y)

            for t_class in classes:
                for index in numerical_indexes:
                    to_quantile_sketch_array(did,X[:,index], y, num_bin, t_class, index)
                
        except:
            print("Error with dataset " + str(did))
            continue;
        
    

In [9]:
# Save the compressed datasets
def save_compressed_dataset(path):
    np.save(path + "compressed.npy", qsa_representation)

In [95]:
# CREATING THE NEURAL NETS

def initialize_MLPs():
    
    for transf in transformations_name:
        model = Sequential()

        model.add(Dense(64, input_shape=inp_shape, W_regularizer=regularizers.l2(0.01)))
        model.add(Activation('softmax'))
        model.add(Dropout(dropout))
        model.add(Dense(64))
        model.add(Flatten())
        model.add(Dense(output_dim=1))
        model.add(Activation('relu'))

        # For a binary classification problem
        model.compile(optimizer='rmsprop',
                      loss='binary_crossentropy',
                      metrics=['accuracy'])

        MLP_LFE_Nets[transf] = model
    

In [65]:

def load_compressed_ds():
    data = pd.DataFrame(np.load("datasets/compressed/compressed.npy"))
    data.columns = [str(i) for i in range (num_bin*2+2)]
    return data


def assemble_training_set(compressed, transformation_targets):
    targetDf = pd.DataFrame(transformation_targets)
    targetDf.columns = ["0","1", "2"]
    merged = pd.merge(compressed, targetDf, how='left', on=["0", "1"])
    class_1 = merged.ix[:,2:num_bin + 2].values
    class_2 = merged.ix[:,num_bin + 2:-1].values
    target = np.array(merged.ix[:,-1].values)
    meta_inf = np.array(merged.ix[:,:2].values)
    meta_target = np.c_[target, meta_inf]
    X = []
    
    for c1, c2 in zip(class_1, class_2):
        X.append([c1,c2])    

    return np.array(X), np.array(meta_target)


def split_training_test():
    compressed_ds = load_compressed_ds()

    for transf, name in zip(transformations, transformations_name):
        transformation_targets = np.load("datasets/compressed/" + name + "2.npy")
        X,y = assemble_training_set(compressed_ds, transformation_targets)
        X_s_tr, X_s_test, y_s_tr, y_s_test = train_test_split(X, y, test_size=0.3)
        
        # Dropping the meta-info from training set
        y_s_tr = y_s_tr[:,:1]
        
        np.save("datasets/training/" + name + "-data_split",X_s_tr)
        np.save("datasets/training/" + name + "-target_split",y_s_tr)
        np.save("datasets/test/" + name + "-data_split",X_s_test)
        np.save("datasets/test/" + name + "-target_split",y_s_test)
 
        
def load_training_set(transf):
    
    X = np.load("datasets/training/" + transf + "-data_split.npy")
    y = np.load("datasets/training/" + transf + "-target_split.npy")
    
    return X,y


In [12]:
def balance_dataset(X, y, pos_perc = 0.5):
    X = np.array(X)
    y = np.array(y)

    cnt = Counter(y)

    neg_num = cnt[0]
    pos_num = cnt[1]

    neg_index = (y == 0)
    pos_index = (y == 1)

    X_pos = X[pos_index]
    X_neg = X[neg_index]
    y_pos = y[pos_index]
    y_neg = y[neg_index]

    if pos_perc >= 0.5:
        if pos_num <= neg_num:
            sample_neg_num = int(X_pos.shape[0] * (1/pos_perc - 1))
            neg_sampled_indexes = np.random.choice(X_neg.shape[0], sample_neg_num, replace=False)
            X_neg = X_neg[neg_sampled_indexes]
            y_neg = y_neg[neg_sampled_indexes]

        else:
            sample_neg_ideal_size = int(X_pos.shape[0] * (1/pos_perc - 1))

            if(sample_neg_ideal_size > X_neg.shape[0]):
                sample_pos_num = int(X_neg.shape[0] * (1/(1-pos_perc) - 1))
                pos_sampled_indexes = np.random.choice(X_pos.shape[0], sample_pos_num, replace=False)
                X_pos = X_pos[pos_sampled_indexes]
                y_pos = y_pos[pos_sampled_indexes]
            else:
                neg_sampled_indexes = np.random.choice(X_neg.shape[0], sample_neg_ideal_size, replace=False) 
                X_neg = X_neg[neg_sampled_indexes]
                y_neg = y_neg[neg_sampled_indexes]
    else:
        if pos_num <= neg_num:
            sample_pos_ideal_size = int(X_neg.shape[0] * (1/(1-pos_perc) - 1))

            if(sample_pos_ideal_size > X_pos.shape[0]):
                sample_neg_num = int(X_pos.shape[0] * (1/pos_perc - 1))
                neg_sampled_indexes = np.random.choice(X_neg.shape[0], sample_neg_num, replace=False)
                X_neg = X_neg[neg_sampled_indexes]
                y_neg = y_neg[neg_sampled_indexes]
            else:
                pos_sampled_indexes = np.random.choice(X_pos.shape[0], sample_pos_ideal_size, replace=False) 
                X_pos = X_pos[pos_sampled_indexes]
                y_pos = y_pos[pos_sampled_indexes]
        else:
            sample_pos_num = int(X_neg.shape[0] * (1/(1-pos_perc) - 1))
            pos_sampled_indexes = np.random.choice(X_pos.shape[0], sample_pos_num, replace=False)
            X_pos = X_pos[pos_sampled_indexes]
            y_pos = y_pos[pos_sampled_indexes]

    X = np.concatenate((X_pos, X_neg), axis=0)
    y = np.concatenate((y_pos, y_neg) , axis=0)      

    shuffle_index = np.random.choice(X.shape[0], X.shape[0], replace=False)

    X = X[shuffle_index]
    y = y[shuffle_index]

    return X,y

In [96]:
# Training the nets

def train_MLPs():

    for transf, name in zip(transformations, transformations_name):
        print("Start training " + name)
        
        X, y = load_training_set(name)
        
        #if(X.shape[0] > train_set_max):
        #        new_indexes = np.random.choice(X.shape[0], too_big, replace=False)
        #        X = X[new_indexes]
        #        y = y[new_indexes]
        
        X, y = balance_dataset(X, y.reshape(y.shape[0]), pos_perc=0.5)
        print("dataset_balanced")
        
        y = y.reshape(y.shape[0],1)

        MLP_LFE_Nets[name].summary()
        print ("Inputs: {}".format(MLP_LFE_Nets[name].input_shape))
        print ("Outputs: {}".format(MLP_LFE_Nets[name].output_shape))
        print ("Actual input: {}".format(X.shape))
        print ("Actual output: {}".format(y.shape))
        
        csv_logger = CSVLogger("datasets/MLPs/log/log-"+ name +".csv", append=True, separator=';')
        MLP_LFE_Nets[name].fit(X, y, epochs=500, verbose=1, validation_split=0.2 ,callbacks=[csv_logger]) 


In [14]:

def save_MLPs():
    for transf in transformations_name:
        MLP_LFE_Nets[transf].save_weights("datasets/MLPs/" + transf + "-weights")
        model_json = MLP_LFE_Nets[transf].to_json()
        with open("datasets/MLPs/" + transf + "-net_model", "w") as f:
            f.write(model_json)

def load_MLPs():
    
    for name in transformations_name:
        json_file = open('datasets/MLPs/' + name + "-net_model", 'r')
        loaded_model_json = json_file.read()
        json_file.close()
        loaded_model = model_from_json(loaded_model_json)
        
        # load weights into new model
        loaded_model.load_weights("datasets/MLPs/" + name + "-weights")

        # evaluate loaded model on test data
        loaded_model.compile(optimizer='rmsprop',
                             loss='binary_crossentropy',
                             metrics=['accuracy'])
        
        MLP_LFE_Nets[name] = loaded_model


In [15]:
def load_test_set():
    
    X = np.load("datasets/test/" + transformations_name[0] + "-data_split.npy")
    y_meta = np.load("datasets/test/" + transformations_name[0] + "-target_split.npy")
    t = np.full((y_meta.shape[0],1), 0)
    y_meta = np.concatenate((y_meta, t), axis=1)
    
    for i,name in enumerate(transformations_name[1:]):
        X = np.concatenate((X, np.load("datasets/test/" + name + "-data_split.npy")), axis=0)
        y_meta_tmp = np.load("datasets/test/" + name + "-target_split.npy")
        t = np.full((y_meta_tmp.shape[0],1),i+1)
        y_meta_tmp = np.concatenate((y_meta_tmp, t), axis=1)
        y_meta = np.concatenate((y_meta, y_meta_tmp), axis=0)
    
    return X,y_meta

In [16]:
def evaluate_transformation_classifier():
    
    # Number of prediction on features
    num_of_prediction = {}
    # Number of correct prediction on features
    num_of_correct_prediction = {}
    # Number of dataset which received a prediction
    good_predicted_dids = set()
    num_of_predicted_dataset = 0
    
    pred_mat = []
    
    X, y_meta = load_test_set()
    
    if(X.shape[0] > test_set_max):
        new_indexes = np.random.choice(X.shape[0], too_big, replace=False)
        X = X[new_indexes]
        y_meta = y_meta[new_indexes]
    
    for transf in transformations_name:
        pred_mat.append(MLP_LFE_Nets[transf].predict(X))
        num_of_prediction[transf] = 0
        num_of_correct_prediction[transf] = 0
        
    pred_mat = np.array(pred_mat).transpose()
    
    for predictions,did,feature in zip(pred_mat[0],y_meta[:,1], y_meta[:,2]):
        pmax = np.amax(predictions)
        print(predictions)
        
        if pmax > pred_threshold:
            index = np.where(predictions==pmax)[0][0]
            print(index)
            num_of_prediction[transformations_name[index]] += 1
            
            # Select the target for the transformation and the dataset
            positive_example_found =  np.where((y_meta[:,0] == 1)       &  \
                                               (y_meta[:,1] == did)     &  \
                                               (y_meta[:,2] == feature) &  \
                                               (y_meta[:,3] == index))     \
                                                                       [0].shape[0] > 0
            
            if(positive_example_found):
                good_predicted_dids.add(did) 
                num_of_correct_prediction[transformations_name[index]] += 1
    
    
    
    for transf in transformations_name:
        
        if(num_of_prediction[transf] == 0):
            print("No predictions have been made")
            continue
    
        print("Evalutation of the transformation classifier: " + transf)
        print("\tNumber of prediction:", num_of_prediction[transf])
        print("\tNumber of Correct prediciton:", num_of_correct_prediction[transf])
        print("Accuracy:", num_of_correct_prediction[transf]/num_of_prediction[transf])
        
    print("\n")    
    print("Number of datasets who received a good prediction:", len(good_predicted_dids))
    print("Total number of positive examples: " + str(np.where(y_meta[:,0] == 1)[0].shape[0]))
    print("Total number of examples: " + str(X.shape[0]))

# Build and Preprocess the Dataset

In [ ]:

build_target_for_compressed(dids)
save_target_for_compressed("datasets/compressed/")
    

In [ ]:

build_compressed_dataset(dids)
save_compressed_dataset("datasets/compressed/")


# Train Test Split

In [125]:

split_training_test()


C:\Users\Andrea\Anaconda3\lib\site-packages\ipykernel_launcher.py:12: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate_ix
  if sys.path[0] == '':



# Train the MLPs


In [126]:

initialize_MLPs()


C:\Users\Andrea\Anaconda3\lib\site-packages\ipykernel_launcher.py:8: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(64, input_shape=(2, 10), kernel_regularizer=<keras.reg...)`
  
C:\Users\Andrea\Anaconda3\lib\site-packages\ipykernel_launcher.py:13: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(units=1)`
  del sys.path[0]


In [129]:

train_MLPs()


Start training tanh
dataset_balanced
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_92 (Dense)             (None, 2, 64)             704       
_________________________________________________________________
activation_77 (Activation)   (None, 2, 64)             0         
_________________________________________________________________
dropout_38 (Dropout)         (None, 2, 64)             0         
_________________________________________________________________
dense_93 (Dense)             (None, 2, 64)             4160      
_________________________________________________________________
flatten_38 (Flatten)         (None, 128)               0         
_________________________________________________________________
dense_94 (Dense)             (None, 1)                 129       
_________________________________________________________________
activation_78 (Activation)   (None, 1) 

48/48 [==============================] - 0s - loss: 1.4087 - acc: 0.7083 - val_loss: 0.2156 - val_acc: 0.7500
Epoch 56/500
48/48 [==============================] - 0s - loss: 1.4344 - acc: 0.6458 - val_loss: 0.2264 - val_acc: 0.7500
Epoch 57/500
48/48 [==============================] - 0s - loss: 1.1119 - acc: 0.6250 - val_loss: 0.2461 - val_acc: 0.7500
Epoch 58/500
48/48 [==============================] - 0s - loss: 1.3915 - acc: 0.6875 - val_loss: 0.2372 - val_acc: 0.7500
Epoch 59/500
48/48 [==============================] - 0s - loss: 1.3917 - acc: 0.6667 - val_loss: 0.2318 - val_acc: 0.7500
Epoch 60/500
48/48 [==============================] - 0s - loss: 1.4158 - acc: 0.7083 - val_loss: 0.2044 - val_acc: 0.7500
Epoch 61/500
48/48 [==============================] - 0s - loss: 1.4564 - acc: 0.6875 - val_loss: 0.1953 - val_acc: 0.7500
Epoch 62/500
48/48 [==============================] - 0s - loss: 1.7915 - acc: 0.6667 - val_loss: 0.2682 - val_acc: 0.6667
Epoch 63/500
48/48 [=========

48/48 [==============================] - 0s - loss: 1.1632 - acc: 0.5833 - val_loss: 0.1408 - val_acc: 0.6667
Epoch 122/500
48/48 [==============================] - 0s - loss: 1.1522 - acc: 0.6458 - val_loss: 0.1329 - val_acc: 0.7500
Epoch 123/500
48/48 [==============================] - 0s - loss: 1.0606 - acc: 0.6250 - val_loss: 0.1379 - val_acc: 0.6667
Epoch 124/500
48/48 [==============================] - 0s - loss: 1.0958 - acc: 0.6042 - val_loss: 0.1402 - val_acc: 0.6667
Epoch 125/500
48/48 [==============================] - 0s - loss: 1.1030 - acc: 0.5208 - val_loss: 0.1160 - val_acc: 0.6667
Epoch 126/500
48/48 [==============================] - 0s - loss: 1.1034 - acc: 0.6458 - val_loss: 0.0963 - val_acc: 0.7500
Epoch 127/500
48/48 [==============================] - 0s - loss: 1.4680 - acc: 0.6667 - val_loss: 0.1167 - val_acc: 0.6667
Epoch 128/500
48/48 [==============================] - 0s - loss: 1.1104 - acc: 0.6875 - val_loss: 0.0941 - val_acc: 0.7500
Epoch 129/500
48/48 [=

48/48 [==============================] - 0s - loss: 1.4381 - acc: 0.6042 - val_loss: 0.1284 - val_acc: 0.8333
Epoch 187/500
48/48 [==============================] - 0s - loss: 1.1384 - acc: 0.5833 - val_loss: 0.1187 - val_acc: 0.8333
Epoch 188/500
48/48 [==============================] - 0s - loss: 0.7756 - acc: 0.6458 - val_loss: 0.1135 - val_acc: 0.8333
Epoch 189/500
48/48 [==============================] - 0s - loss: 1.0498 - acc: 0.6250 - val_loss: 0.1195 - val_acc: 0.8333
Epoch 190/500
48/48 [==============================] - 0s - loss: 1.0806 - acc: 0.6458 - val_loss: 0.1156 - val_acc: 0.8333
Epoch 191/500
48/48 [==============================] - 0s - loss: 1.5342 - acc: 0.6042 - val_loss: 0.1008 - val_acc: 0.8333
Epoch 192/500
48/48 [==============================] - 0s - loss: 1.4640 - acc: 0.7708 - val_loss: 0.1070 - val_acc: 0.8333
Epoch 193/500
48/48 [==============================] - 0s - loss: 1.1242 - acc: 0.7500 - val_loss: 0.1128 - val_acc: 0.8333
Epoch 194/500
48/48 [=

48/48 [==============================] - 0s - loss: 1.0624 - acc: 0.7708 - val_loss: 0.0681 - val_acc: 0.8333
Epoch 253/500
48/48 [==============================] - 0s - loss: 0.8116 - acc: 0.7083 - val_loss: 0.0756 - val_acc: 0.8333
Epoch 254/500
48/48 [==============================] - 0s - loss: 1.0917 - acc: 0.7500 - val_loss: 0.0756 - val_acc: 0.8333
Epoch 255/500
48/48 [==============================] - 0s - loss: 0.7864 - acc: 0.7708 - val_loss: 0.0858 - val_acc: 0.8333
Epoch 256/500
48/48 [==============================] - 0s - loss: 1.1149 - acc: 0.6875 - val_loss: 0.0821 - val_acc: 0.8333
Epoch 257/500
48/48 [==============================] - 0s - loss: 1.0658 - acc: 0.6875 - val_loss: 0.0776 - val_acc: 0.8333
Epoch 258/500
48/48 [==============================] - 0s - loss: 1.0541 - acc: 0.7083 - val_loss: 0.0747 - val_acc: 0.8333
Epoch 259/500
48/48 [==============================] - 0s - loss: 1.4056 - acc: 0.6667 - val_loss: 0.0761 - val_acc: 0.8333
Epoch 260/500
48/48 [=

...

14/14 [==============================] - 0s - loss: 0.0261 - acc: 0.6429 - val_loss: 0.0213 - val_acc: 0.5000
Epoch 216/500
14/14 [==============================] - 0s - loss: 0.0213 - acc: 0.5714 - val_loss: 0.0212 - val_acc: 0.5000
Epoch 217/500
14/14 [==============================] - 0s - loss: 0.0212 - acc: 0.5714 - val_loss: 0.0212 - val_acc: 0.5000
Epoch 218/500
14/14 [==============================] - 0s - loss: 0.0212 - acc: 0.5714 - val_loss: 0.0211 - val_acc: 0.5000
Epoch 219/500
14/14 [==============================] - 0s - loss: 0.0621 - acc: 0.5714 - val_loss: 0.0213 - val_acc: 0.5000
Epoch 220/500
14/14 [==============================] - 0s - loss: 0.0474 - acc: 0.7143 - val_loss: 0.0213 - val_acc: 0.5000
Epoch 221/500
14/14 [==============================] - 0s - loss: 0.0554 - acc: 0.5714 - val_loss: 0.0212 - val_acc: 0.5000
Epoch 222/500
14/14 [==============================] - 0s - loss: 0.0212 - acc: 0.5000 - val_loss: 0.0212 - val_acc: 0.5000
Epoch 223/500
14/14 [=

14/14 [==============================] - 0s - loss: 0.0201 - acc: 0.5714 - val_loss: 0.0201 - val_acc: 0.5000
Epoch 282/500
14/14 [==============================] - 0s - loss: 0.0320 - acc: 0.5714 - val_loss: 0.0201 - val_acc: 0.5000
Epoch 283/500
14/14 [==============================] - 0s - loss: 0.0201 - acc: 0.6429 - val_loss: 0.0201 - val_acc: 0.5000
Epoch 284/500
14/14 [==============================] - 0s - loss: 0.0201 - acc: 0.5000 - val_loss: 0.0200 - val_acc: 0.5000
Epoch 285/500
14/14 [==============================] - 0s - loss: 0.0319 - acc: 0.5714 - val_loss: 0.0200 - val_acc: 0.5000
Epoch 286/500
14/14 [==============================] - 0s - loss: 0.0200 - acc: 0.5000 - val_loss: 0.0200 - val_acc: 0.5000
Epoch 287/500
14/14 [==============================] - 0s - loss: 0.0200 - acc: 0.5000 - val_loss: 0.0199 - val_acc: 0.5000
Epoch 288/500
14/14 [==============================] - 0s - loss: 0.0199 - acc: 0.5000 - val_loss: 0.0199 - val_acc: 0.5000
Epoch 289/500
14/14 [=

14/14 [==============================] - 0s - loss: 0.0192 - acc: 0.5000 - val_loss: 0.0192 - val_acc: 0.5000
Epoch 348/500
14/14 [==============================] - 0s - loss: 0.0349 - acc: 0.5714 - val_loss: 0.0193 - val_acc: 0.5000
Epoch 349/500
14/14 [==============================] - 0s - loss: 0.0193 - acc: 0.5714 - val_loss: 0.0192 - val_acc: 0.5000
Epoch 350/500
14/14 [==============================] - 0s - loss: 0.0192 - acc: 0.6429 - val_loss: 0.0192 - val_acc: 0.5000
Epoch 351/500
14/14 [==============================] - 0s - loss: 0.0192 - acc: 0.5000 - val_loss: 0.0191 - val_acc: 0.5000
Epoch 352/500
14/14 [==============================] - 0s - loss: 0.0191 - acc: 0.5000 - val_loss: 0.0191 - val_acc: 0.5000
Epoch 353/500
14/14 [==============================] - 0s - loss: 0.0191 - acc: 0.5000 - val_loss: 0.0190 - val_acc: 0.5000
Epoch 354/500
14/14 [==============================] - 0s - loss: 0.0265 - acc: 0.5714 - val_loss: 0.0191 - val_acc: 0.5000
Epoch 355/500
14/14 [=

14/14 [==============================] - 0s - loss: 0.0190 - acc: 0.5714 - val_loss: 0.0190 - val_acc: 0.5000
Epoch 414/500
14/14 [==============================] - 0s - loss: 0.0437 - acc: 0.5714 - val_loss: 0.0190 - val_acc: 0.5000
Epoch 415/500
14/14 [==============================] - 0s - loss: 0.0259 - acc: 0.5714 - val_loss: 0.0191 - val_acc: 0.5000
Epoch 416/500
14/14 [==============================] - 0s - loss: 0.0191 - acc: 0.5000 - val_loss: 0.0190 - val_acc: 0.5000
Epoch 417/500
14/14 [==============================] - 0s - loss: 0.0217 - acc: 0.5714 - val_loss: 0.0190 - val_acc: 0.5000
Epoch 418/500
14/14 [==============================] - 0s - loss: 0.0190 - acc: 0.5000 - val_loss: 0.0190 - val_acc: 0.5000
Epoch 419/500
14/14 [==============================] - 0s - loss: 0.0190 - acc: 0.5000 - val_loss: 0.0190 - val_acc: 0.5000
Epoch 420/500
14/14 [==============================] - 0s - loss: 0.0190 - acc: 0.5000 - val_loss: 0.0190 - val_acc: 0.5000
Epoch 421/500
14/14 [=

14/14 [==============================] - 0s - loss: 0.0187 - acc: 0.5000 - val_loss: 0.0186 - val_acc: 0.5000
Epoch 480/500
14/14 [==============================] - 0s - loss: 0.0808 - acc: 0.5000 - val_loss: 0.0187 - val_acc: 0.5000
Epoch 481/500
14/14 [==============================] - 0s - loss: 0.0187 - acc: 0.5714 - val_loss: 0.0187 - val_acc: 0.5000
Epoch 482/500
14/14 [==============================] - 0s - loss: 0.0814 - acc: 0.4286 - val_loss: 0.0187 - val_acc: 0.5000
Epoch 483/500
14/14 [==============================] - 0s - loss: 0.0187 - acc: 0.5714 - val_loss: 0.0186 - val_acc: 0.5000
Epoch 484/500
14/14 [==============================] - 0s - loss: 0.0186 - acc: 0.5000 - val_loss: 0.0186 - val_acc: 0.5000
Epoch 485/500
14/14 [==============================] - 0s - loss: 0.0186 - acc: 0.5714 - val_loss: 0.0186 - val_acc: 0.5000
Epoch 486/500
14/14 [==============================] - 0s - loss: 0.0186 - acc: 0.5000 - val_loss: 0.0186 - val_acc: 0.5000
Epoch 487/500
14/14 [=

In [131]:

save_MLPs()


# Load the nets

In [123]:

load_MLPs()



# Test the nets


In [130]:

evaluate_transformation_classifier()


[ 0.98939091  0.          0.3031953 ]
0
[ 1.86476457  0.          2.58061719]
2
[ 0.  0.  0.]
[ 0.  0.  0.]
[ 0.  0.  0.]
[ 0.  0.  0.]
[ 0.  0.  0.]
[ 0.  0.  0.]
[ 1.31659424  0.          0.        ]
0
[ 0.45552471  0.          0.        ]
[ 0.  0.  0.]
[ 0.87769789  0.          0.01699541]
0
[ 1.05090499  0.          0.39866728]
0
[ 0.70834374  0.          2.97649908]
2
[ 0.  0.  0.]
[ 1.70888913  0.          0.        ]
0
[ 1.76332927  0.          0.        ]
0
[ 0.  0.  0.]
[ 0.  0.  0.]
[ 0.  0.  0.]
[ 0.  0.  0.]
[ 1.02276063  0.          2.87714291]
2
[ 0.  0.  0.]
[ 0.1437815  0.         0.       ]
[ 0.58688331  0.          3.15172696]
2
[ 1.70888913  0.          0.        ]
0
[ 0.  0.  0.]
[ 0.  0.  0.]
[ 1.70888913  0.          0.        ]
0
[ 0.89294869  0.          2.86900902]
2
[ 0.  0.  0.]
[ 0.1114895   0.          2.39087439]
2
[ 0.14664854  0.          0.3695904 ]
[ 0.26781464  0.          0.38834459]
[ 0.         0.         0.2968868]
[ 0.76336908  0.          2.9390

In [ ]:
pred_threshold = 0.55
